In [29]:
import os
import pandas as pd
import geopandas as gpd
import shapely.wkb
from shapely.geometry import Point
from sqlalchemy import create_engine, text

In [2]:
from dotenv import load_dotenv

load_dotenv()

DBNAME = os.getenv('DBNAME')
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')
CONNEXION = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}"

PATH_DATA_G = os.getenv('PATH_DATA_G')
G_FILOSOFI = 'filosofi'

### CODE / ECHELLE

In [ ]:
# Création de l'engine SQLAlchemy
engine = create_engine(CONNEXION)

requete = text(
    """
    select 
    numero_siren::int as code,
    'epci' as echelle
    from hab.v_epcifp_500 
    union 
    select insee_com::int as code,
    'commune' as echelle
    from hab.commune_500 
    where insee_com like '34%' 
    order by code
    """
    )

# Tester la connexion
with engine.connect() as connection:
    result = connection.execute(requete)
    code_com_epci = pd.DataFrame(result.fetchall(), columns=result.keys())



DBNAME_ANCIEN, USER_ANCIEN, PASSWORD_ANCIEN, HOST_ANCIEN, PORT_ANCIEN = "obs", "hab", "34!hab99", "s934", "5441"
CONNEXION_ANCIEN = f"postgresql+psycopg2://{USER_ANCIEN}:{PASSWORD_ANCIEN}@{HOST_ANCIEN}:{PORT_ANCIEN}/{DBNAME_ANCIEN}"

engine_ancien = create_engine(CONNEXION_ANCIEN)

requete = text(
    """
    select inseeiris::int as code,
    'iris' as echelle 
    from hab.iris_100 
    order by code
    """
    )

# Tester la connexion
with engine_ancien.connect() as connection:
    result = connection.execute(requete)
    code_iris = pd.DataFrame(result.fetchall(), columns=result.keys())

code = pd.concat([code_com_epci, code_iris], axis=0).drop_duplicates().reset_index(drop=True)
code['echelle'].value_counts()

,code,echelle
0,34001,commune
1,34002,commune
2,34003,commune
3,34004,commune
4,34005,commune
...,...,...
931,343420000,iris
932,343430000,iris
933,343440101,iris
934,343440102,iris


### ZONAGE GEOGRAPHQUE

In [58]:
# Création de l'engine SQLAlchemy
engine = create_engine(CONNEXION)

requete = text(
    """
    select *
    from hab.v_epcifp_500 
    """
    )

# Tester la connexion
with engine.connect() as connection:
    result = connection.execute(requete)
    zonage_epci = pd.DataFrame(result.fetchall(), columns=result.keys())


requete = text(
    """
    select *
    from hab.commune_500 
    where insee_com like '34%' 
    """
    )

# Tester la connexion
with engine.connect() as connection:
    result = connection.execute(requete)
    zonage_com = pd.DataFrame(result.fetchall(), columns=result.keys())

DBNAME_ANCIEN, USER_ANCIEN, PASSWORD_ANCIEN, HOST_ANCIEN, PORT_ANCIEN = "obs", "hab", "34!hab99", "s934", "5441"
CONNEXION_ANCIEN = f"postgresql+psycopg2://{USER_ANCIEN}:{PASSWORD_ANCIEN}@{HOST_ANCIEN}:{PORT_ANCIEN}/{DBNAME_ANCIEN}"

engine_ancien = create_engine(CONNEXION_ANCIEN)

requete = text(
    """
    select * 
    from hab.iris_100 
    """
    )

# Tester la connexion
with engine_ancien.connect() as connection:
    result = connection.execute(requete)
    rows = result.fetchall()
    columns = result.keys()
    dict_rows = [dict(zip(columns, row)) for row in rows]
    for row in dict_rows:
        row['shape'] = shapely.wkb.loads(row['shape'], hex=True)
    zonage_iris = gpd.GeoDataFrame(dict_rows, geometry='shape', crs="EPSG:2154")
    zonage_iris = zonage_iris.to_crs(epsg=4326) # WGS 84 -> Lambert 93
    zonage_iris['inseeiris'] = zonage_iris['inseeiris'].astype(int)


## DECRET

In [ ]:
G_DECRET = 'decret'
chemin = os.path.join(PATH_DATA_G, G_DECRET, "2023_decret.xlsx")
decret = pd.read_excel(chemin, sheet_name="3")

## AIRDNA

In [60]:
G_AIRDNA = 'airdna'

In [15]:
print("Chargement des données Airbnb...")
chemin = os.path.join(PATH_DATA_G, G_AIRDNA, 'offre_airdna_34_2022.xlsx')
bnb2022 = pd.read_excel(chemin)
bnb2022 = bnb2022[bnb2022['Liste logement']=='Entire home/apt']
bnb2022 = bnb2022[['INSEE','Communes','Code EPCI','EPCI','Longitude','Latitude','Revenu annuel €',
                   'Nb jours réservés','Nb total jours de disponibilité']]

chemin = os.path.join(PATH_DATA_G, G_AIRDNA, 'offre_airdna_34_2023.xlsx')
bnb2023 = pd.read_excel(chemin)
bnb2023 = bnb2023[bnb2023['Nature logement']=='Entire home/apt']
bnb2023 = bnb2023[['INSEE','Communes','Code EPCI','EPCI','Longitude','Latitude','Revenu annuel â‚¬',
                             'Nb jours rÃ©servÃ©s','Nb total jours de disponibilitÃ©']]
bnb2023.columns = ['INSEE','Communes','Code EPCI','EPCI','Longitude','Latitude','Revenu annuel €',
                             'Nb jours réservés','Nb total jours de disponibilité']
print("Chargement des données Airbnb terminé.")

Chargement des données Airbnb...
Chargement des données Airbnb terminé.


In [61]:
bnb2022['geometry'] = bnb2022.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
bnb2022_gdf = gpd.GeoDataFrame(bnb2022, geometry='geometry', crs="EPSG:4326")
bnb2022_gdf = gpd.sjoin(bnb2022_gdf, zonage_iris, how="left", predicate="intersects")

In [62]:
bnb2023['geometry'] = bnb2023.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
bnb2023_gdf = gpd.GeoDataFrame(bnb2023, geometry='geometry', crs="EPSG:4326")
bnb2023_gdf = gpd.sjoin(bnb2023_gdf, zonage_iris, how="left", predicate="intersects")

In [63]:
bnb2023_gdf[['inseeiris', 'INSEE', 'Code EPCI', 
                'Nb jours réservés', 'Revenu annuel €', 'Nb total jours de disponibilité']]

,inseeiris,INSEE,Code EPCI,Nb jours réservés,Revenu annuel €,Nb total jours de disponibilité
0,342900000.0,34290,200022986,20.0,5052.0,37.0
1,343000000.0,34300,243400769,78.0,2543.0,132.0
2,340390000.0,34039,200066355,69.0,14488.0,209.0
3,340030201.0,34003,243400819,16.0,1019.0,54.0
5,340030201.0,34003,243400819,73.0,5846.0,114.0
...,...,...,...,...,...,...
46092,343370103.0,34337,243400017,33.0,15874.0,63.0
46093,343010601.0,34301,200066355,27.0,6200.0,54.0
46094,341540105.0,34154,243400470,0.0,0.0,0.0
46095,340030301.0,34003,243400819,96.0,29213.0,117.0


In [54]:
col_a_garder = ['inseeiris', 'INSEE', 'Code EPCI', 'Nb jours réservés', 'Revenu annuel €', 'Nb total jours de disponibilité']
bnb = pd.merge(bnb2022_gdf[col_a_garder], bnb2023_gdf[col_a_garder],
                    on=['inseeiris', 'INSEE', 'Code EPCI'], 
                    how='outer', suffixes=('_2022', '_2023'))
bnb

,inseeiris,INSEE,Code EPCI,Nb jours réservés_2022,Revenu annuel €_2022,Nb total jours de disponibilité_2022,Nb jours réservés_2023,Revenu annuel €_2023,Nb total jours de disponibilité_2023
0,340010000,34001,200071058,158.0,7406.0,164.0,175.0,7616.0,317.0
1,340010000,34001,200071058,158.0,7406.0,164.0,45.0,13366.0,204.0
2,340010000,34001,200071058,158.0,7406.0,164.0,0.0,0.0,0.0
3,340010000,34001,200071058,158.0,7406.0,164.0,149.0,25128.0,232.0
4,340010000,34001,200071058,158.0,7406.0,164.0,11.0,1150.0,19.0
...,...,...,...,...,...,...,...,...,...
32562531,NaN,34344,243400470,76.0,4911.0,115.0,0.0,0.0,0.0
32562532,NaN,34344,243400470,76.0,4911.0,115.0,6.0,758.0,33.0
32562533,NaN,34344,243400470,76.0,4911.0,115.0,6.0,542.0,6.0
32562534,NaN,34344,243400470,76.0,4911.0,115.0,15.0,2119.0,25.0


In [ ]:
result = pd.DataFrame()

def aggregate_bnb(df, code_bnb):
    
    return df.groupby(code_bnb).agg(
        nombre_reserv_120=('Nb jours réservés', lambda x: (x >= 120).sum()),
        nombre_dispo_120=('Nb total jours de disponibilité', lambda x: (x >= 120).sum()),
        nombre_total=('Nb jours réservés', 'count'),
        jours_reserves=('Nb jours réservés', 'sum'),
        revenu=('Revenu annuel €', 'sum'),
        jours_reserves_120=('Nb jours réservés', lambda x: x[df.loc[x.index, 'Nb total jours de disponibilité'] >= 120].sum()),
        revenu_120=('Revenu annuel €', lambda x: x[df.loc[x.index, 'Nb total jours de disponibilité'] >= 120].sum())
    ).reset_index()


for code_bnb, echelle in zip(['inseeiris', 'INSEE', 'Code EPCI'], ['iris', 'commune', 'epci']):
    bnb2022_agg = aggregate_bnb(bnb2022_gdf, code_bnb)
    bnb2023_agg = aggregate_bnb(bnb2023_gdf, code_bnb)

    bnb = pd.merge(bnb2022_agg, bnb2023_agg, 
                   on=code_bnb, how='outer', suffixes=('_2022', '_2023'))

    code = pd.merge(code, bnb, left_on='code', right_on=code_bnb, how='left')






       inseeiris  nombre_reserv_120_2022  nombre_dispo_120_2022  \
0    340010000.0                     2.0                    4.0   
1    340020000.0                     2.0                   10.0   
2    340030101.0                    20.0                   57.0   
3    340030102.0                     8.0                   26.0   
4    340030103.0                     6.0                   25.0   
..           ...                     ...                    ...   
571  343420000.0                     0.0                    3.0   
572  343430000.0                     0.0                    1.0   
573  343440101.0                   152.0                  347.0   
574  343440102.0                   280.0                  650.0   
575  343440103.0                    39.0                  100.0   

     nombre_total_2022  jours_reserves_2022  revenu_2022  \
0                 16.0                690.0      74625.0   
1                 15.0               1028.0     177214.0   
2              

,code,echelle,inseeiris,nombre_reserv_120_2022_x,nombre_dispo_120_2022_x,nombre_total_2022_x,jours_reserves_2022_x,revenu_2022_x,jours_reserves_120_2022_x,revenu_120_2022_x,...,revenu_2022,jours_reserves_120_2022,revenu_120_2022,nombre_reserv_120_2023,nombre_dispo_120_2023,nombre_total_2023,jours_reserves_2023,revenu_2023,jours_reserves_120_2023,revenu_120_2023
0,34001,commune,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34002,commune,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34003,commune,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34004,commune,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34005,commune,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,343420000,iris,343420000.0,0.0,3.0,9.0,273.0,41789.0,15745.0,15745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932,343430000,iris,343430000.0,0.0,1.0,20.0,683.0,105182.0,15745.0,15745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
933,343440101,iris,343440101.0,152.0,347.0,706.0,52884.0,4553978.0,15745.0,15745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
934,343440102,iris,343440102.0,280.0,650.0,1220.0,98197.0,8813928.0,15745.0,15745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# iris = gpd.read_file("données/couches/georef-herault-iris.geojson")

# Convertir bnb2022 en GeoDataFrame et effectuer la jointure spatiale avec iris
bnb2022['geometry'] = bnb2022.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
bnb2022_gdf = gpd.GeoDataFrame(bnb2022, geometry='geometry', crs="EPSG:4326")
bnb2022_gdf = gpd.sjoin(bnb2022_gdf, zonage_iris, how="left", predicate="intersects") # , op="intersects"

# Sélectionner les colonnes d'intérêt
bnb2022_gdf = bnb2022_gdf[['code_officiel_iris', 'code_officiel_commune', 'Code EPCI', 
                        'Nb jours réservés', 'Revenu annuel €', 'Nb total jours de disponibilité']]

# Agréger les données
bnb2022_agg = bnb2022_gdf.groupby(['code_officiel_iris', 'code_officiel_commune']).agg(
    nombre_reserv_120=('Nb jours réservés', lambda x: (x >= 120).sum()),
    nombre_dispo_120=('Nb total jours de disponibilité', lambda x: (x >= 120).sum()),
    nombre_total=('Nb jours réservés', 'count'),
    jours_reserves=('Nb jours réservés', 'sum'),
    revenu=('Revenu annuel €', 'sum'),
    jours_reserves_120=('Nb jours réservés', lambda x: (bnb2022_gdf['Nb total jours de disponibilité'] >= 120).sum()),
    revenu_120=('Revenu annuel €', lambda x: (bnb2022_gdf['Nb total jours de disponibilité'] >= 120).sum())
).reset_index()

# Convertir en DataFrame classique
bnb2022_agg = pd.DataFrame(bnb2022_agg)

# Convertir les colonnes Latitude et Longitude de bnb2023 en valeurs numériques
bnb2023['Latitude'] = pd.to_numeric(bnb2023['Latitude'], errors='coerce')
bnb2023['Longitude'] = pd.to_numeric(bnb2023['Longitude'], errors='coerce')

# Filtrer les observations avec des valeurs non manquantes
bnb2023 = bnb2023.dropna(subset=['Latitude', 'Longitude'])

# Convertir bnb2023 en GeoDataFrame et effectuer la jointure spatiale avec iris
bnb2023['geometry'] = bnb2023.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
bnb2023_gdf = gpd.GeoDataFrame(bnb2023, geometry='geometry', crs="EPSG:4326")
bnb2023_gdf = gpd.sjoin(bnb2023_gdf, iris, how="left", predicate="intersects")

# Sélectionner les colonnes d'intérêt
bnb2023_gdf = bnb2023_gdf[['code_officiel_iris', 'code_officiel_commune', 
                        'Nb jours réservés', 'Revenu annuel €', 'Nb total jours de disponibilité']]

# Agréger les données
bnb2023_agg = bnb2023_gdf.groupby(['code_officiel_iris', 'code_officiel_commune']).agg(
    nombre_reserv_120=('Nb jours réservés', lambda x: (x >= 120).sum()),
    nombre_dispo_120=('Nb total jours de disponibilité', lambda x: (x >= 120).sum()),
    nombre_total=('Nb jours réservés', 'count'),
    jours_reserves=('Nb jours réservés', 'sum'),
    revenu=('Revenu annuel €', 'sum'),
    jours_reserves_120=('Nb jours réservés', lambda x: (bnb2023_gdf['Nb total jours de disponibilité'] >= 120).sum()),
    revenu_120=('Revenu annuel €', lambda x: (bnb2023_gdf['Nb total jours de disponibilité'] >= 120).sum())
).reset_index()

# Convertir en DataFrame classique
bnb2023_agg = pd.DataFrame(bnb2023_agg)

# Fusionner les tables pour comparer les années 2022 et 2023
table_iris = pd.merge(bnb2022_agg, bnb2023_agg, 
                    on=['code_officiel_iris', 'code_officiel_commune'], 
                    how='outer', suffixes=('_2022', '_2023'))

# Calculer les évolutions et les écarts
table_iris = table_iris.assign(
    evol_bnb_120=(table_iris['nombre_dispo_120_2023'] - table_iris['nombre_dispo_120_2022']) / table_iris['nombre_dispo_120_2022'],
    ecart_bnb_120=table_iris['nombre_dispo_120_2023'] - table_iris['nombre_dispo_120_2022'],
    ecart_bnb_tot=table_iris['nombre_total_2023'] - table_iris['nombre_total_2022'],
    evol_bnb_tot=(table_iris['nombre_total_2023'] - table_iris['nombre_total_2022']) / table_iris['nombre_total_2022'],
    ecart_jours=table_iris['jours_reserves_2023'] - table_iris['jours_reserves_2022'],
    ecart_revenu=table_iris['revenu_2023'] - table_iris['revenu_2022'],
    ecart_revenu_jours=(table_iris['revenu_2023'] / table_iris['jours_reserves_2023']) - (table_iris['revenu_2022'] / table_iris['jours_reserves_2022'])
)

In [ ]:
def build_bnb_com(df):
    """
    Variables:
        df: pd.DataFrame
    Entrée: pd.DataFrame
    Sortie: pd.DataFrame avec ['Communes', 'INSEE', 'Nb jours réservés', 'Nb total jours de disponibilité', 'Revenu annuel €'] comme colonnes
    """
    return df.groupby(['Communes', 'INSEE']).agg(
        nombre_reserv_120=('Nb jours réservés', lambda x: (x >= 120).sum()),
        nombre_dispo_120=('Nb total jours de disponibilité', lambda x: (x >= 120).sum()),
        nombre_total=('Nb jours réservés', 'count'),
        jours_reserves=('Nb jours réservés', 'sum'),
        revenu=('Revenu annuel €', 'sum'),
        jours_reserves_120=('Nb jours réservés', lambda x: ((df.loc[x.index, 'Nb total jours de disponibilité'] >= 120) * x).sum()),
        revenu_120=('Revenu annuel €', lambda x: ((df.loc[x.index, 'Nb total jours de disponibilité'] >= 120) * x).sum())
    ).reset_index()


# Partie 1: Airbnb ----
bnb2022_com = build_bnb_com(bnb2022)
bnb2023_com = build_bnb_com(bnb2023)    
# Fusion des deux DataFrames
bnb_com = pd.merge(bnb2023_com, bnb2022_com, on=['INSEE'], suffixes=('_2023', '_2022'))

# Ajout des colonnes calculées
bnb_com = bnb_com.assign(
    evol_bnb_120=(bnb_com['nombre_dispo_120_2023'] - bnb_com['nombre_dispo_120_2022']) / bnb_com['nombre_dispo_120_2022'],
    ecart_bnb_120=bnb_com['nombre_dispo_120_2023'] - bnb_com['nombre_dispo_120_2022'],
    ecart_bnb_tot=bnb_com['nombre_total_2023'] - bnb_com['nombre_total_2022'],
    evol_bnb_tot=(bnb_com['nombre_total_2023'] - bnb_com['nombre_total_2022']) / bnb_com['nombre_total_2022'],
    ecart_jours=bnb_com['jours_reserves_2023'] - bnb_com['jours_reserves_2022'],
    ecart_revenu=bnb_com['revenu_2023'] - bnb_com['revenu_2022'],
    ecart_revenu_jours=(bnb_com['revenu_2023'] / bnb_com['jours_reserves_2023']) - (bnb_com['revenu_2022'] / bnb_com['jours_reserves_2022'])
)

bnb_com['evol_bnb_1202'] = bnb_com.apply(
    lambda row: 0.01 if row['nombre_dispo_120_2022'] == 0 else (row['nombre_dispo_120_2023'] - row['nombre_dispo_120_2022']) / row['nombre_dispo_120_2022'],
    axis=1
)


In [ ]:
# Airbnb data processing
bnb2022_com = build_bnb_com(bnb2022)
bnb2023_com = build_bnb_com(bnb2023)


bnb_com = pd.merge(bnb2023_com, bnb2022_com, on="INSEE", suffixes=("_2023", "_2022"))

bnb_com['INSEE'] = bnb_com['INSEE'].astype(str)
corres_com_epci['INSEE'] = corres_com_epci['INSEE'].astype(str)
table_epci = pd.merge(bnb_com, corres_com_epci, on="INSEE")

table_epci = table_epci.groupby('code_epci').agg(
    nombre_dispo_120_2022=('nombre_dispo_120_2022', 'sum'),
    nombre_dispo_120_2023=('nombre_dispo_120_2023', 'sum'),
    nombre_reserv_120_2022=('nombre_reserv_120_2022', 'sum'),
    nombre_reserv_120_2023=('nombre_reserv_120_2023', 'sum'),
    nombre_total_2022=('nombre_total_2022', 'sum'),
    nombre_total_2023=('nombre_total_2023', 'sum'),
    jours_reserves_2022=('jours_reserves_2022', 'sum'),
    jours_reserves_2023=('jours_reserves_2023', 'sum'),
    revenu_2022=('revenu_2022', 'sum'),
    revenu_2023=('revenu_2023', 'sum')
).reset_index()

table_epci['evol_bnb_120'] = (table_epci['nombre_dispo_120_2023'] - table_epci['nombre_dispo_120_2022']) / table_epci['nombre_dispo_120_2022']
table_epci['ecart_bnb_120'] = table_epci['nombre_dispo_120_2023'] - table_epci['nombre_dispo_120_2022']
table_epci['ecart_bnb_tot'] = table_epci['nombre_total_2023'] - table_epci['nombre_total_2022']
table_epci['evol_bnb_tot'] = (table_epci['nombre_total_2023'] - table_epci['nombre_total_2022']) / table_epci['nombre_total_2022']
table_epci['ecart_jours'] = table_epci['jours_reserves_2023'] - table_epci['jours_reserves_2022']
table_epci['ecart_revenu'] = table_epci['revenu_2023'] - table_epci['revenu_2022']
table_epci['ecart_revenu_jours'] = (table_epci['revenu_2023'] / table_epci['jours_reserves_2023']) - (table_epci['revenu_2022'] / table_epci['jours_reserves_2022'])


## FILOSOFI

In [ ]:
chemin = os.path.join(PATH_DATA_G, G_FILOSOFI, 'filosofi_com_2020.xlsx')
filosofi_com_2020 = pd.read_excel(chemin, sheet_name=0)
chemin = os.path.join(PATH_DATA_G, G_FILOSOFI, 'filosofi_epci_2020.xlsx')
filosofi_epci_2020 = pd.read_excel(chemin)


filosofi = pd.concat([filosofi_com_2020, filosofi_epci_2020], ignore_index=True)

table = pd.merge(code_com_epci, filosofi, how='left', left_on="code", right_on="CODGEO")
table = table.drop(columns=['CODGEO'])